# Piperack Generator

## Module imports and connection to STAAD

In [1]:
from base.helper import *
from base.staad_base.geometry import *
from base.structural_elements.beam import *
from base.structural_elements.column import *
from base.piperack.portal import *
from base.staad_base.load import *
from base.staad_base.design import *
from base.staad_base.property import *
from base.staad_base.optimise_member import *
from IPython.display import display, Markdown
from pyperclip import copy

openSTAAD,STAAD_objects = get_openSTAAD()

geometry = STAAD_objects.geometry
property = STAAD_objects.property
output = STAAD_objects.output

add_beams = add_beams(geometry=geometry)

nodes = get_node_incidences(geometry=geometry)
beam_objects:dict = get_beam_objects(geometry=geometry,property=None)
beam_nos:list = list(beam_objects.keys())

## Inputs

In [2]:
base_point_of_first_portal = Point3D(0,0,0)
width_of_piperack = 8
portal_to_portal_distance = [0,8,8,6,6,7,8]
columns = [0,width_of_piperack]
tiers = [3,6,9,12,14]
long_beams = [4.5,7.5,10.5,tiers[-1]]
braces_placement = [False,True,False,False,True,False]

## Generator Code

In [ ]:
portals = []
longitudinal_beams = []
stubs = []
intermediate_transverse_beams = []
intermediate_long_beams = []
plan_braces = []
vertical_braces = []

portal_beam_ids = [] 
portal_column_ids = [] 
long_beam_ids = [] 
stub_ids = []
intermediate_transverse_ids = []
intermediate_long_ids = []
plan_brace_ids = []
vertical_brace_ids = []

portal_count = len(portal_to_portal_distance)
portal = PiperackPortal(base=base_point_of_first_portal) 

for column in columns:
    portal.add_column(Column3D(base=Point3D(column,0,0),height=tiers[-1])) \

for tier in tiers:
    for column_i in range(len(columns)-1):
        portal.add_beam(Beam3D(start=Point3D(columns[column_i],tier,0),end=Point3D(columns[column_i+1],tier,0))) 

portals = [portal.shift(Point3D(0,0,sum(portal_to_portal_distance[:i]))) for i in range(0,portal_count)]

for portal_i in range(len(portals)-1):
    for column in columns:
        for long_beam in long_beams:
            long_beam_x = Beam3D(start=Point3D(column,long_beam,portals[portal_i].base.z),end=Point3D(column,long_beam,portals[portal_i+1].base.z))
            longitudinal_beams.append(long_beam_x)


for portal in portals:
    portal_beam_ids = [*portal_beam_ids,*add_beams(portal.beams)]
    portal_column_ids = [*portal_column_ids,*add_beams(portal.columns)]

long_beam_ids = add_beams(longitudinal_beams)

markdown_output = ''
markdown_output = "## Primary Members Created\n\n"
markdown_output += "| Member Types | IDs | \n"
markdown_output += "| --- | --- |\n"
markdown_output += f"| Portal Beams | {portal_beam_ids} |\n"
markdown_output += f"| Portal Columns | {portal_column_ids} |\n"
markdown_output += f"| Long Beams | {long_beam_ids} |\n"

display(Markdown(markdown_output))

## Primary Members Created

| Member Types | IDs | 
| --- | --- |
| Portal Beams | [1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31, 32, 33, 36, 37, 38, 39, 40] |
| Portal Columns | [6, 7, 6, 7, 13, 14, 20, 21, 27, 28, 34, 35, 41, 42] |
| Long Beams | [0, 0, 0, 0, 0, 0, 0, 0, 50, 52, 54, 55, 57, 59, 61, 62, 64, 66, 68, 69, 71, 73, 75, 76, 78, 80, 82, 83, 85, 87, 89, 90, 92, 94, 96, 97, 99, 101, 103, 104, 106, 108, 110, 111, 113, 115, 117, 118] |


In [4]:
long_beams_y = group_beams_by_y(longitudinal_beams)

previous_y = None
for y,group_y in long_beams_y.items():
    base = portal.base.y if not previous_y else previous_y
    long_beams_sorted = beams_sorted_yxz(group_y)

    for beam_i in range(len(long_beams_sorted)-1):
        portal_i = beam_i%(len(portals)-1)
        if(braces_placement[portal_i]):
            long_beam_i : Beam3D = long_beams_sorted[beam_i]

            vb_1 = Beam3D(  start=Point3D(long_beam_i.start.x,
                                        base,
                                        long_beam_i.start.z),
                            end=long_beam_i.mid())
            vb_2 = Beam3D(  start=Point3D(long_beam_i.start.x,
                                        base,
                                        long_beam_i.end.z),
                            end=long_beam_i.mid())
            vertical_braces.append(vb_1)
            vertical_braces.append(vb_2)
    previous_y = y

vertical_brace_ids = add_beams(vertical_braces)

create_stub = lambda beamA,beamB : Beam3D(start=beamA.mid(),end=beamB.mid())

for long_beam_i in range(len(longitudinal_beams)-1):
    long_ii:Beam3D = longitudinal_beams[long_beam_i]
    long_ij:Beam3D = longitudinal_beams[long_beam_i+1]
    if(long_ii.start.eq_x(long_ij.start) and long_ii.end.eq_z(long_ij.end)):
        stubs.append(create_stub(long_ii,long_ij))

stubs = beams_sorted_yzx(stubs)

for stub_i in range(0, len(stubs), 2):
    if stub_i + 1 < len(stubs):  
        stub1, stub2 = stubs[stub_i], stubs[stub_i + 1]
        intermediate_transverse_beams.append(Beam3D(start=stub1.mid(),end=stub2.mid()))

stub_ids = add_beams(stubs)
intermediate_transverse_ids = add_beams(intermediate_transverse_beams)

portal_beams_y = group_beams_by_y([beam for portal in portals for beam in portal.beams])
for group_y in portal_beams_y.values():
    sorted_beams = beams_sorted_yzx(group_y)
    for beam_i in range(len(sorted_beams)-1):
        beam1 = sorted_beams[beam_i]
        beam2 = sorted_beams[beam_i+1]
        intermediate_long_beams.append(Beam3D(start=beam1.mid(),end=beam2.mid()))

        if(braces_placement[beam_i]):
            plan_braces.append(Beam3D(start=beam1.start,end=beam2.end))
            plan_braces.append(Beam3D(start=beam2.start,end=beam1.end))


intermediate_long_ids = add_beams(intermediate_long_beams)
plan_brace_ids = add_beams(plan_braces)

markdown_output = ''
markdown_output = "## Secondary Members Created\n\n"
markdown_output += "| Member Types | IDs | \n"
markdown_output += "| --- | --- |\n"
markdown_output += f"| Stubs | {stub_ids} |\n"
markdown_output += f"| Intermediate Trans Beams | {intermediate_transverse_ids} |\n"
markdown_output += f"| Intermediate Long Beams | {intermediate_long_ids} |\n"
markdown_output += f"| Plan Braces | {plan_brace_ids} |\n"
markdown_output += f"| Vertical Braces | {vertical_brace_ids} |\n"

display(Markdown(markdown_output))

## Secondary Members Created

| Member Types | IDs | 
| --- | --- |
| Stubs | [43, 46, 167, 168, 171, 174, 177, 180, 181, 182, 185, 188, 44, 47, 189, 190, 192, 194, 196, 198, 199, 200, 202, 204, 45, 48, 205, 206, 208, 210, 212, 214, 215, 216, 218, 220] |
| Intermediate Trans Beams | [2, 223, 226, 229, 232, 235, 3, 238, 241, 244, 247, 250, 253, 256, 259, 262, 265, 268] |
| Intermediate Long Beams | [0, 271, 273, 275, 277, 279, 0, 282, 284, 286, 288, 290, 0, 293, 295, 297, 299, 301, 0, 304, 306, 308, 310, 312, 0, 315, 317, 319, 321, 323] |
| Plan Braces | [324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343] |
| Vertical Braces | [120, 121, 123, 124, 126, 127, 129, 130, 132, 133, 135, 136, 138, 139, 141, 142, 144, 145, 147, 148, 150, 151, 153, 154, 156, 157, 159, 160, 162, 163, 165, 166] |
